# Working with MPAS Meshes

##  Mesh Definition

All components of MPAS lie on a C-grid staggered Voronoi Mesh as seen in the figure below:

<p align="center">
  <img src="../_static/examples/mpas/c-grid.png"
  width="400" / >
</p>

This type of mesh is represented through two different sub-meshes. The Primal Mesh represents the Voronoi Mesh and the Dual Mesh represents the Triangular Mesh.

*to-do: more in-depth description/overview of MPAS from the website and specification document.*




## Imports

In [49]:
import xarray as xr
import uxarray as ux

## Grid and Data Files

As mentioned in earlier notebooks, the grid definition and data variables are typically stored in separate files. However, in this example, our dataset will contain both within the same file, which is often the case when working with smaller datasets.



In [50]:
mpas_root_filepath = "../../test/meshfiles/mpas/"
mpas_dataset_filepath = mpas_root_filepath + "ice-sheet/gis_20km_r01.20210824.nc"

In [51]:
mpas_xr_ds = xr.open_dataset(mpas_dataset_filepath)
mpas_xr_ds

<xarray.Dataset>
Dimensions:                  (Time: 1, nCells: 7425, nEdges: 22660,
                              nVertices: 15236, TWO: 2, maxEdges: 6,
                              maxEdges2: 12, vertexDegree: 3, nVertLevels: 10)
Dimensions without coordinates: Time, nCells, nEdges, nVertices, TWO, maxEdges,
                                maxEdges2, vertexDegree, nVertLevels
Data variables: (12/43)
    muFriction               (Time, nCells) float64 ...
    effectivePressure        (nCells) float64 ...
    latCell                  (nCells) float64 ...
    lonCell                  (nCells) float64 ...
    xCell                    (nCells) float64 ...
    yCell                    (nCells) float64 ...
    ...                       ...
    layerThicknessFractions  (nVertLevels) float64 ...
    thickness                (Time, nCells) float64 ...
    temperature              (Time, nCells, nVertLevels) float64 ...
    bedTopography            (nCells) float64 ...
    sfcMassBal               (nCells) float64 ...
    beta                     (nCells) float64 ...
Attributes:
    on_a_sphere:               NO
    sphere_radius:             0.0
    parent_id:                 vvvb3zzrse
    history:                   Tue Dec 14 12:10:32 2021: ncap2 -s muFriction=...
    mesh_spec:                 0.0
    Conventions:               MPAS
    source:                    MpasCellCuller.x
    mesh_id:                   zvzfm0exax
    NCO:                       netCDF Operators version 4.7.4 (http://nco.sf....
    nco_openmp_thread_number:  1

This dataset contains the full set of connectivity variables needed to describe an MPAS mesh, as described in the MPAS Specification Document [2]. It also contains data variables such as temperature and thickness, which lie on the Primal Mesh. Below is a list of connectivity variables and their geometric meaning, which will be further described when comparing to the UGRID encoding:


### Primal Mesh
* **lonVertex, latVertex**: Corner Vertices of Primal Mesh cells
* **lonCell, latCell**: Center Vertices of Primal Mesh cells
* **verticesOnCell**: Vertex indices that surround each Primal Mesh cell
* **verticesOnEdge**: Vertex indices that saddle a given edge
* **nEdgesOnCell**: Maximum number of edges that can surround a cell

### Dual Mesh
* **lonCell, latCell**: Corner Vertices of Dual Mesh cells
* **lonVertex, latVertex**: Center Vertices of Dual Mesh cells
* **cellsOnVertex**: Vertex indices that surround each Dual Mesh cell
* **cellsOnEdge**: Vertex indices that saddle a given edge


## Constructing a Grid Object

The `xarray.Dataset` that we opened above stores the coordinates and connectivity variables for both the Primal and Dual meshes together in a single dataset. With `uxarray`, we can create separate `Grid` objects for either the Primal or Dual meshes. The `uxarray.Grid` class parses our input dataset, detects that it is represented in the MPAS specifications, and encodes either the Primal or Dual mesh in the UGRID conventions.

In [52]:
primal_grid = ux.Grid(mpas_xr_ds, use_dual=False)
dual_grid = ux.Grid(mpas_xr_ds, use_dual=True)

In [53]:
primal_grid.ds

<xarray.Dataset>
Dimensions:           (nMesh2_node: 15236, nMesh2_face: 7425,
                       nMaxMesh2_face_nodes: 6, nMesh2_edge: 22660, Two: 2)
Dimensions without coordinates: nMesh2_node, nMesh2_face, nMaxMesh2_face_nodes,
                                nMesh2_edge, Two
Data variables:
    Mesh2             float64 nan
    Mesh2_node_x      (nMesh2_node) float64 -45.75 -45.41 ... -39.39 -37.83
    Mesh2_node_y      (nMesh2_node) float64 59.77 59.79 59.81 ... 83.26 83.26
    Mesh2_face_x      (nMesh2_face) float64 -45.77 -45.43 ... -39.38 -37.85
    Mesh2_face_y      (nMesh2_face) float64 59.87 59.89 59.91 ... 83.16 83.16
    Mesh2_face_nodes  (nMesh2_face, nMaxMesh2_face_nodes) uint64 10 11 ... 15235
    Mesh2_edge_nodes  (nMesh2_edge, Two) uint64 10 11 11 0 ... 15223 15225 15235
Attributes:
    sphere_radius:  0.0
    mesh_id:        zvzfm0exax
    mesh_spec:      0.0
    on_a_sphere:    NO

In [54]:
dual_grid.ds

<xarray.Dataset>
Dimensions:           (nMesh2_node: 7425, nMesh2_face: 15236, Three: 3,
                       nMesh2_edge: 22660, Two: 2)
Dimensions without coordinates: nMesh2_node, nMesh2_face, Three, nMesh2_edge,
                                Two
Data variables:
    Mesh2             float64 nan
    Mesh2_node_x      (nMesh2_node) float64 -45.77 -45.43 ... -39.38 -37.85
    Mesh2_node_y      (nMesh2_node) float64 59.87 59.89 59.91 ... 83.16 83.16
    Mesh2_face_x      (nMesh2_face) float64 -45.75 -45.41 ... -39.39 -37.83
    Mesh2_face_y      (nMesh2_face) float64 59.77 59.79 59.81 ... 83.26 83.26
    Mesh2_face_nodes  (nMesh2_face, Three) uint64 18446744073709551615 ... 18...
    Mesh2_edge_nodes  (nMesh2_edge, Two) uint64 0 ... 18446744073709551615
Attributes:
    sphere_radius:  0.0
    mesh_id:        zvzfm0exax
    mesh_spec:      0.0
    on_a_sphere:    NO

## Relationship between MPAS and UGRID
*to-do: describe MPAS to UGRID encoding, differences in fill values, standardized datatypes, and how geometric connectivity variables relate between these two conventions*

## References

[1] https://mpas-dev.github.io/

[2] https://mpas-dev.github.io/files/documents/MPAS-MeshSpec.pdf

[3] http://ugrid-conventions.github.io/ugrid-conventions/